### 1-Import Libraries

In [ ]:
import os, datetime, warnings, logging, random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import keras,cv2
from PIL import Image
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from keras import layers
from keras import models
from keras import optimizers
from tqdm.notebook import tqdm
import glob, random, time, shutil
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPool2D, BatchNormalization, Activation, Input, Add, Dense, ZeroPadding2D,Flatten, AveragePooling2D, Rescaling, Dropout, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras import metrics
from joblib import Parallel, delayed
from sklearn.preprocessing import label_binarize, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback,ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from keras_tuner import RandomSearch

In [ ]:
#loading the model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetV2_preprocessing
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionV3_preprocessing

### 2-Fix randomness and hide warnings

In [ ]:
# Fix randomness and hide warnings
SEED = 42

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

np.random.seed(SEED)

import logging

random.seed(SEED)

### 3-Set first parameters of tensorflow

In [ ]:
# Import tensorflow
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
print("tensorflow_version",tf.__version__)

### 4-Read DATA

In [ ]:
dataset = np.load("/kaggle/input/a2ndl-farm/public_data.npz", allow_pickle=True)

### 5-Data Inspection

In [ ]:
keys = list(dataset.keys())
print('keys in our dataset are: ', keys)

#data shape
images = dataset["data"]
no_images = images.shape[0]
size_images = images.shape[1:3]
print('Data shape: ',images.shape)

#labels
labels = dataset["labels"]
no_labels = labels.shape[0]

#check balanced or imbalanced of DATA
unique_classes, counts = np.unique(labels,return_counts=True)
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} data points")

### 6_Show images

In [ ]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the string labels to integer labels
integer_labels = label_encoder.fit_transform(labels)

# Convert integers to binary labels (0 or 1)
binary_labels = np.where(integer_labels == 1, 1, 0)

In [ ]:
def show_images(dataset, labels, batch_no, no_images_per_batch, num_cols=10):

    start_index = batch_no * no_images_per_batch
    end_index = min((batch_no + 1) * no_images_per_batch, len(dataset))
    num_rows = (end_index - start_index + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))

    for i, ax in enumerate(axes.ravel()):
        if start_index + i < end_index:
            image = dataset[start_index + i]
            image = image / 255.0  # Normalize pixel values to [0, 1]
            ax.imshow(image)
            ax.set_title(f"{labels[start_index + i]}",fontsize=10, y=-0.5)
            ax.axis('off')

    for i in range(end_index - start_index, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    plt.show()

In [ ]:
show_images(dataset=images,labels =binary_labels, batch_no=0,no_images_per_batch=60)

### 7-Remove Outliers

In [ ]:
# there are some uselees images in dataset which must be first recognized and then remoeved.

def histogram_similarity(image, reference_histogram, threshold=0.8):

    # Calculate the color histogram of the image
    image_histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    cv2.normalize(image_histogram, image_histogram)
    intersection = cv2.compareHist(reference_histogram, image_histogram, cv2.HISTCMP_INTERSECT)

    return intersection < threshold


# Load a reference histogram from a non-Shrek-Singer image
reference_image = images[0]  # Use the first image as a reference which it's a leaf image
reference_histogram = cv2.calcHist([reference_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
cv2.normalize(reference_histogram, reference_histogram)

similarity_threshold = 0.62
odd_images = []

# Iterate through the images in data_array
for image in images:
    if histogram_similarity(image, reference_histogram, similarity_threshold):
        odd_images.append(image)

In [ ]:
#Creating reference images
reference_unwanted_images = []

# Define the index of the image we want to plot and save as a reference
image_index_trololo = 10  # Replace with the index of the reference image
image_index_shrek = 33

if 0 <= image_index_trololo  and image_index_shrek< len(odd_images):
    # Get the image based on the index
    trololo_reference_image = odd_images[image_index_trololo]
    shrek_reference_image = odd_images[image_index_shrek]

    reference_unwanted_images.append(trololo_reference_image)
    reference_unwanted_images.append(shrek_reference_image)

else:
    print("Invalid image index.")

In [ ]:
from skimage.color import rgb2lab, deltaE_cie76

#Calculate the color distance between two images using the CIE76 metric.
def calculate_color_distance(image1, image2):

    lab_image1 = rgb2lab(image1)
    lab_image2 = rgb2lab(image2)

    return deltaE_cie76(lab_image1, lab_image2)

def find_unwanted_images_by_color_distance(data_array, reference_unwanted_images, color_distance_threshold):

    cleaned_data = []
    unwanted_images = []
    labels_index =[]
    outlier_labels_index=[]
    index =0
    for image in data_array:
        similar = False
        for reference_image in reference_unwanted_images:
            color_distance = calculate_color_distance(image, reference_image)
            if (color_distance < color_distance_threshold).all():
                similar = True
                break  # No need to check further if a match is found
        if (similar == True):
            unwanted_images.append(image)
            labels_index.append(index)
        else:
            cleaned_data.append(image)
            outlier_labels_index.append(index)

        index+=1

    return cleaned_data,unwanted_images, labels_index

color_distance_threshold = 1

In [ ]:
cleaned_images,unwanted_images, labels_index = find_unwanted_images_by_color_distance(images, reference_unwanted_images, color_distance_threshold)

In [ ]:
unwanted_images = np.stack(unwanted_images, axis=0) # Define new shape for unwanted_images
unwanted_images.shape

In [ ]:
cleaned_images = np.stack(cleaned_images, axis=0)
cleaned_images.shape

In [ ]:
# Create a boolean mask for the values to keep
mask = np.ones(binary_labels.shape[0], dtype=bool)
mask[labels_index] = False

cleaned_labels = binary_labels[mask]
outlier_labels=binary_labels[labels_index]

In [ ]:
print(" "*25,end=" ")
print("Unwanted Images ",end=' '*25)
show_images(dataset=unwanted_images,labels=outlier_labels, batch_no=0,no_images_per_batch=10)

In [ ]:
print(" "*25,end=" ")
print("Cleaned Images ",end=' '*25)
show_images(dataset=cleaned_images,labels =cleaned_labels, batch_no=0,no_images_per_batch=10)

In [ ]:
no_images = cleaned_images.shape[0]
no_labels = cleaned_labels.shape[0]
_, counts = np.unique(cleaned_labels,return_counts=True) # count occurrence of each item
no_healthy_images = counts[0]
no_unhealthy_images = counts[1]

# pass variables to a dictionary to be used as dataframe for a better show
info_table_dict = {"no. images":no_images, "image width": size_images[0],"image length": size_images[1], "no. labels":no_labels,
                   "no. healthy_images":no_healthy_images,"percentage%":no_healthy_images*100/no_labels,
                   "no. unhealthy_images":no_unhealthy_images,"percentage %":no_unhealthy_images*100/no_labels }
print(" "*42,end=" ")
print("Table after Removing Outliers ",end=' '*42)
info_table = pd.DataFrame(info_table_dict, index =['value'])
info_table

# 8-Class Weight

In [ ]:
# Calculate class frequencies
class_frequencies = np.sum(np.array(category_labels), axis=0)

# Calculate class weights inversely proportional to class frequencies
total_samples = np.sum(class_frequencies)
class_weights = {i: total_samples / (len(class_frequencies) * freq) for i, freq in enumerate(class_frequencies)}

# 9-Splitting data And Augmentation

In [ ]:
BATCH_SIZE = 64
category_labels = to_categorical(cleaned_labels, num_classes=2)

In [ ]:
# Constructor for training data generator with augmentation
data_datagen = ImageDataGenerator(
    rotation_range=30,
#     height_shift_range=0.2,
#     width_shift_range=0.2,
    zoom_range=[0.7 , 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='reflect',
    rescale=1./255,
    # change this based on the network you want use!!!
    # default : CNN e.g. 'resnet_preprocess'
    validation_split=0.1,
    preprocessing_function=None
)

# # Constructor for validation data generator without augmentation comment for TTA
# data_datagen_val = ImageDataGenerator(
#     rescale=1./255,
#     validation_split=0.1,  # Use the same validation split
#     preprocessing_function=None
# )

# Generator for training set
aug_train_set = data_datagen.flow(
    cleaned_images,
    category_labels,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    subset="training"
)

# Generator for validation set (with augmentation)
validation_set = data_datagen.flow(
    cleaned_images,
    category_labels,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    subset="validation"
)

# 10-Define Models

**CNN**

In [ ]:
def CNN_model(hp):
    model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                                 kernel_size=hp.Choice('conv1_kernel', values=[3, 5]), activation='elu', input_shape=(96, 96, 3)),
          tf.keras.layers.BatchNormalization(renorm=True),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=128, step=32),
                                 kernel_size=hp.Choice('conv2_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.Conv2D(filters=hp.Int('conv3_filters', min_value=64, max_value=256, step=32),
                                 kernel_size=hp.Choice('conv3_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.BatchNormalization(renorm=True),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(filters=hp.Int('conv4_filters', min_value=64, max_value=512, step=32),
                                 kernel_size=hp.Choice('conv4_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.Conv2D(filters=hp.Int('conv5_filters', min_value=128, max_value=1024, step=64),
                                 kernel_size=hp.Choice('conv5_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.BatchNormalization(renorm=True),
          tf.keras.layers.Conv2D(filters=hp.Int('conv6_filters', min_value=64, max_value=512, step=64),
                                 kernel_size=hp.Choice('conv6_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.Conv2D(filters=hp.Int('conv7_filters', min_value=32, max_value=256, step=16),
                                 kernel_size=hp.Choice('conv7_kernel', values=[3, 5]), activation='elu'),
          tf.keras.layers.GlobalMaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dropout(rate=hp.Float('dropout1_rate', min_value=0.05, max_value=0.2, step=0.05),seed=SEED),
          tf.keras.layers.Dense(units=hp.Int('dense1_units', min_value=64, max_value=512, step=32),activation= None),
          tf.keras.layers.Dropout(rate=hp.Float('dropout2_rate', min_value=0.4, max_value=0.6, step=0.1),seed=SEED),
          tf.keras.layers.Dense(units=hp.Int('dense2_units', min_value=32, max_value=256, step=16),activation= None),
          tf.keras.layers.Dense(units=hp.Int('dense3_units', min_value=32, max_value=128, step=8),
                                activation=keras.layers.LeakyReLU(alpha=hp.Float('leaky_relu_alpha', min_value=0.01, max_value=0.3, step=0.01))),
          tf.keras.layers.Dense(2, activation='softmax')
          ])

    opt = tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4]))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Utility function for callbacks for training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 5 , verbose=1,factor=0.3, min_lr=0.000001)
# Model checkpoint
mcp_save_CNN = ModelCheckpoint('model_CNN.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# history_CNN_simple = model_CNN.fit(aug_train_set,
#     epochs=50,
#     verbose=1,
#     callbacks=[learning_rate_reduction, mcp_save_CNN],
#     validation_data= validation_set
# )

In [ ]:
# history_CNN_deleteoutliers = model_CNN.fit(aug_train_set,
#     epochs=50,
#     verbose=1,
#     callbacks=[learning_rate_reduction, mcp_save_CNN],
#     validation_data= validation_set
# )

In [ ]:
history_CNN_Aug = loaded_model.fit(aug_train_set,
    epochs=50,
    verbose=1,
    callbacks=[learning_rate_reduction, mcp_save_CNN],
    validation_data= validation_set
)

In [ ]:
tuner = RandomSearch(
    CNN_model,
    objective='val_accuracy',
    max_trials=10,  # تعداد تلاش‌ها برای جستجو
    directory='/kaggle/working',  # مسیر ذخیره سازی نتایج
    project_name='cnn_tuning'  # نام پروژه
)

In [ ]:
tuner.search(aug_train_set,
    epochs=50,
    verbose=1,
    callbacks=[learning_rate_reduction, mcp_save_CNN],
    validation_data= validation_set,
    class_weight=class_weights
)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.save('/kaggle/working/SubmissionModel')

from shutil import make_archive
make_archive('/kaggle/working/SubmissionModel_CNN', 'zip', '/kaggle/working/SubmissionModel')

In [ ]:
# with open(BASE_DIR+'history_CNN_Aug.pkl', 'wb') as file:
#     pickle.dump(history_CNN_Aug.history, file)
# with open(BASE_DIR+'history_CNN_simple.pkl', 'rb') as file:
#     history_CNN_simple = pickle.load(file)
# with open(BASE_DIR+'history_CNN_deleteoutliers.pkl', 'rb') as file:
#     history_CNN_deleteoutliers = pickle.load(file)
# with open(BASE_DIR+'history_CNN_Aug.pkl', 'rb') as file:
#     history_CNN_Aug = pickle.load(file)

In [ ]:
# #plotting the validation and train loss
# plt.plot(history_CNN_simple['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
# plt.plot(history_CNN_simple['val_loss'], label='CNN', alpha=.8, color='#ff7f0e')
# plt.plot(history_CNN_deleteoutliers['loss'], alpha=.3, color='#4D61E2', linestyle='--')
# plt.plot(history_CNN_deleteoutliers['val_loss'], label='CNN_Clean', alpha=.8, color='#4D61E2')
# plt.plot(history_CNN_Aug['loss'], alpha=.3, color='#4de2a1', linestyle='--')
# plt.plot(history_CNN_Aug['val_loss'], label='CNN_Aug', alpha=.8, color='#4de2a1')
# plt.title('Categorical Crossentropy')
# plt.grid(alpha=.3)
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(loc='upper left')
# plt.savefig('Categorical Crossentropy.jpg')
# plt.show()

In [ ]:
# #plotting the validation and train ACC
# plt.plot(history_CNN_simple['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
# plt.plot(history_CNN_simple['val_accuracy'], label='simple CNN', alpha=.8, color='#ff7f0e')
# plt.plot(history_CNN_deleteoutliers['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
# plt.plot(history_CNN_deleteoutliers['val_accuracy'], label='CNN_Clean', alpha=.8, color='#4D61E2')
# plt.plot(history_CNN_Aug['accuracy'], alpha=.3, color='#4de2a1', linestyle='--')
# plt.plot(history_CNN_Aug['val_accuracy'], label='CNN_Aug', alpha=.8, color='#4de2a1')
# plt.title('model accuracy')
# plt.grid(alpha=.3)
# plt.legend(loc='upper left')
# plt.savefig('Model Accuracy.jpg')
# plt.show()

**MobileNET**

In [ ]:
# Create MobileNetV2 model with specified settings
input_shape = (96, 96, 3)
mobile = tfk.applications.MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
# tfk.utils.plot_model(mobile, show_shapes=True)

In [ ]:
# Create an input layer based on your input shape
input_layer = tfkl.Input(shape=input_shape)

# Connect MobileNetV2 to the input layer
x = mobile(input_layer)

x = tfkl.Dropout(0.5)(x)
x = tfkl.Dense(512, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))(x)
x = tfkl.Dropout(0.5)(x)
x = tfkl.Dense(128, activation='leaky_relu', kernel_initializer = tf.keras.initializers.HeUniform(SEED))(x)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax' , kernel_initializer=keras.initializers.HeUniform(SEED) )(x)

# Create a Model connecting input and output
MobileNetV2_model = tf.keras.Model(inputs=input_layer, outputs=outputs, name='model')

# make it true for FT-
MobileNetV2_model.get_layer('mobilenetv2_1.00_96').trainable = False

# for i, layer in enumerate(MobileNetV2_model.get_layer('mobilenetv2_1.00_96').layers[:51]):
#   layer.trainable=False

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
MobileNetV2_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display model summary
MobileNetV2_model.summary()

In [ ]:
# Utility function for callbacks for training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 10, verbose=1,factor=0.2, min_lr=0.000001)
# Model checkpoint
mcp_save_model_mobile = ModelCheckpoint('MobileNetV2_model.hdf5', save_best_only=True, monitor='accuracy', mode='max')

In [ ]:
# Train the model
history_MobileNetV2_model = MobileNetV2_model.fit(
    aug_train_set,
    steps_per_epoch=len(aug_train_set),
    epochs=50,
    verbose=1,
    callbacks=[learning_rate_reduction, mcp_save_model_mobile],
    validation_data= validation_set,
#     class_weight=class_weights
)

In [ ]:
#plotting the validation and train loss
plt.plot(history_MobileNetV2_model.history['loss'])
plt.plot(history_MobileNetV2_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the validation and train ACC
plt.plot(history_MobileNetV2_model.history['accuracy'])
plt.plot(history_MobileNetV2_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('Model Accuracy_MobileNet.jpg')
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

true_labels = category_labels[validation_set.index_array]
true_labels = true_labels.astype(int)

prediction =MobileNetV2_model.predict(validation_set)
binary_predictions = (prediction > 0.5).astype(int)

cm = classification_report(true_labels, binary_predictions)
print(cm)

In [ ]:
MobileNetV2_model.save('/kaggle/working/SubmissionModel')

from shutil import make_archive
make_archive('/kaggle/working/SubmissionModel_MobileNet', 'zip', '/kaggle/working/SubmissionModel')

**ResNet50**

In [ ]:
def resnet50_model(Hyparam):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
    GAP = layers.GlobalAveragePooling2D()
    flatten_layer = tf.keras.layers.Flatten()
    dropout = tf.keras.layers.Dropout(0.2)
    dense_layer = layers.Dense(Hyparam, activation='relu', kernel_initializer = tf.keras.initializers.HeUniform(SEED))
    dropout_1 = tf.keras.layers.Dropout(0.5)
    prediction_layer = layers.Dense(2, activation='softmax')
    batch_norm = layers.BatchNormalization()
    Relu = layers.ReLU()
    for layer in base_model.layers[:40]:
        layer.trainable = False
    model = Sequential([
    base_model,
    GAP,
    dropout,
    dense_layer,
    batch_norm,
    dropout_1,
    prediction_layer
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model_resnet50=resnet50_model(128)
model_resnet50.summary()

In [ ]:
# Utility function for callbacks for training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.3, min_lr=0.000001)
# Model checkpoint
mcp_save_ResNet50 = ModelCheckpoint('model_ResNet50.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
history_ResNet50 = model_resnet50.fit(
    aug_train_set,
    steps_per_epoch=len(aug_train_set),
    epochs=50,
    verbose=1,
    callbacks=[learning_rate_reduction, mcp_save_ResNet50],
    validation_data= validation_set
#     class_weight=class_weights
)

In [ ]:
#plotting the validation and train loss
plt.plot(history_ResNet50.history['loss'])
plt.plot(history_ResNet50.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the validation and train ACC
plt.plot(history_ResNet50.history['accuracy'])
plt.plot(history_ResNet50.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('Model Accuracy_ResNet.jpg')
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

true_labels = category_labels[validation_set.index_array]
true_labels = true_labels.astype(int)

prediction =model_resnet50.predict(validation_set)
binary_predictions = (prediction > 0.5).astype(int)

cm = classification_report(true_labels, binary_predictions)
print(cm)

In [ ]:
model_resnet50.save('/kaggle/working/SubmissionModel')

from shutil import make_archive
make_archive('/kaggle/working/SubmissionModel_ResNet50', 'zip', '/kaggle/working/SubmissionModel')

**EfficientnetB0**

In [ ]:
def efficientnetB0_model(Hyparam,Hyparam_1):
    # Load the pre-trained EfficientNetB0 model with weights from ImageNet
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
    GAP = layers.GlobalAveragePooling2D()
    flatten_layer = tf.keras.layers.Flatten()
    dropout = tf.keras.layers.Dropout(0.2,seed=SEED)
    dense_layer = layers.Dense(Hyparam, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))
    dropout_1 = tf.keras.layers.Dropout(0.5,seed=SEED)
    dense_layer_1 = layers.Dense(Hyparam_1, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))
    dropout_2 = tf.keras.layers.Dropout(0.1,seed=SEED)
    prediction_layer = layers.Dense(2, activation='softmax')
    batch_norm = layers.BatchNormalization()
    batch_norm_1 = layers.BatchNormalization()
    Relu = layers.ReLU()
    for layer in base_model.layers[:90]:
        layer.trainable = False
    model = Sequential([
    base_model,
    GAP,
    dropout,
    dense_layer,
    batch_norm,
    dropout_1,
    dense_layer_1,
    batch_norm_1,
    dropout_2,
    prediction_layer
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model_efficientnetB0 = efficientnetB0_model(512,256)
model_efficientnetB0.summary()

In [ ]:
# Utility function for callbacks for training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 5, verbose=1,factor=0.3, min_lr=0.000001)
# Model checkpoint
mcp_save_EfficientnetB0 = ModelCheckpoint('model_efficientnetB0.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
history_efficientnetB0 = model_efficientnetB0.fit(
    aug_train_set,
    steps_per_epoch=len(aug_train_set),
    epochs=60,
    verbose=1,
    callbacks=[mcp_save_EfficientnetB0, mcp_save_ResNet50],
    validation_data= validation_set,
)

In [ ]:
#plotting the validation and train loss
plt.plot(history_efficientnetB0.history['loss'])
plt.plot(history_efficientnetB0.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the validation and train ACC
plt.plot(history_efficientnetB0.history['accuracy'])
plt.plot(history_efficientnetB0.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('Model Accuracy_Efficientnet.jpg')
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

true_labels = category_labels[validation_set.index_array]
true_labels = true_labels.astype(int)

prediction =model_efficientnetB0.predict(validation_set)
binary_predictions = (prediction > 0.5).astype(int)

cm = classification_report(true_labels, binary_predictions)
print(cm)

In [ ]:
model_efficientnetB0.save('/kaggle/working/SubmissionModel')

from shutil import make_archive
make_archive('/kaggle/working/SubmissionModel_Efficien', 'zip', '/kaggle/working/SubmissionModel')

**InceptionV3**

In [ ]:
# Create InceptionV3 model with specified settings
input_shape = (96, 96, 3)
mobile = tfk.applications.InceptionV3(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
# tfk.utils.plot_model(mobile, show_shapes=True)

In [ ]:
# Create an input layer based on your input shape
input_layer = tfkl.Input(shape=input_shape)

# Connect MobileNetV2 to the input layer
x = mobile(input_layer)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(512, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.5)(x)
x = tfkl.Dense(128, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))(x)
x = tfkl.Dropout(0.1)(x)
x = tfkl.Dense(64, activation=None, kernel_initializer = tf.keras.initializers.HeUniform(SEED))(x)


# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
InceptionV3_model = tf.keras.Model(inputs=input_layer, outputs=outputs, name='model')

InceptionV3_model.get_layer('inception_v3').trainable = True
for i, layer in enumerate(InceptionV3_model.get_layer('inception_v3').layers[:38]):
  layer.trainable=False

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
InceptionV3_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display model summary
InceptionV3_model.summary()

In [ ]:
# Utility function for callbacks for training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 5, verbose=1,factor=0.3, min_lr=0.000001)
# Model checkpoint
mcp_save_InceptionV3 = ModelCheckpoint('InceptionV3_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# Train the model
history_InceptionV3_model = InceptionV3_model.fit(
    aug_train_set,
    epochs=60,
    verbose=1,
    callbacks=[learning_rate_reduction, mcp_save_InceptionV3],
    validation_data= validation_set,
    class_weight=class_weights
)

In [ ]:
#plotting the validation and train loss
plt.plot(history_InceptionV3_model.history['loss'])
plt.plot(history_InceptionV3_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the validation and train ACC
plt.plot(history_InceptionV3_model.history['accuracy'])
plt.plot(history_InceptionV3_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('Model Accuracy_InceptionV3.jpg')
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

true_labels = category_labels[validation_set.index_array]
true_labels = true_labels.astype(int)

prediction =InceptionV3_model.predict(validation_set)
binary_predictions = (prediction > 0.5).astype(int)

cm = classification_report(true_labels, binary_predictions)
print(cm)

In [ ]:
InceptionV3_model.save('/kaggle/working/SubmissionModel')

from shutil import make_archive
make_archive('/kaggle/working/SubmissionModel_InceptionV3', 'zip', '/kaggle/working/SubmissionModel')